In [7]:
# 0) Imports & settings
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras import Sequential, Input
from keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from scipy.stats import norm
warnings.filterwarnings('ignore'); plt.rcParams['figure.figsize']=(6,4)
import tensorflow as tf          #  ← add this near the other imports



In [2]:

# 1) Load CSV
df = pd.read_csv('option data variable.csv', parse_dates=['date','exdate'])
df.dropna(inplace=True);  df['strike_price'] /= 1_000

# 2) Feature engineering
df['mid_price']   = (df['best_bid'] + df['best_offer'])/2
df['days_to_exp'] = (df['exdate'] - df['date']).dt.days
df['is_call']     = (df['cp_flag']=='C').astype(int)
df['log_mny']     = np.log(df['underlying_price']/df['strike_price'])
df['log_mny2']    = df['log_mny']**2          # curvature term

X_COLS = ['underlying_price','strike_price','impl_volatility',
          'risk_free_rate','days_to_exp','is_call','log_mny','log_mny2']
Y_COLS = ['mid_price','delta','gamma','vega','theta']

df = df.dropna(subset=X_COLS + Y_COLS).sort_values('date').reset_index(drop=True)


In [3]:

# 3) Chronological split (calls / puts → 98/1/1)
def chrono_split(g, frac=.98):
    n=len(g); i1=int(n*frac); i2=i1+(n-i1)//2
    return g.iloc[:i1], g.iloc[i1:i2], g.iloc[i2:]
call_tr,call_val,call_te = chrono_split(df[df.is_call==1])
put_tr, put_val, put_te  = chrono_split(df[df.is_call==0])


In [4]:

# 4) Scaling helpers
X_scaler = StandardScaler().fit(pd.concat([call_tr,put_tr])[X_COLS])
y_scal_call = StandardScaler().fit(call_tr[Y_COLS])
y_scal_put  = StandardScaler().fit(put_tr[Y_COLS])

def prep(g, xs, ys): return xs.transform(g[X_COLS]), ys.transform(g[Y_COLS])
cXtr,cYtr = prep(call_tr,X_scaler,y_scal_call)
cXva,cYva = prep(call_val,X_scaler,y_scal_call)
cXte,cYte = prep(call_te ,X_scaler,y_scal_call)
pXtr,pYtr = prep(put_tr ,X_scaler,y_scal_put )
pXva,pYva = prep(put_val,X_scaler,y_scal_put )
pXte,pYte = prep(put_te ,X_scaler,y_scal_put )


In [5]:

# 5) Model factory (deeper + Γ head)
def build_mlp(indim, base_units=512, base_layers=5, dropout=.25):
    x = Input(shape=(indim,))
    h = Dense(base_units)(x); h = LeakyReLU()(h)
    for _ in range(base_layers-1):
        h = Dense(base_units)(h); h=BatchNormalization()(h)
        h = LeakyReLU()(h); h = Dropout(dropout)(h)
    common = h
    # shared outputs: price, Δ, ν, θ  (indices 0,1,3,4)
    shared_out = Dense(4, name='shared')(common)
    # separate tiny head for Γ (index 2)
    gamma_out  = Dense(1, name='gamma_head')(common)
    out = tf.keras.layers.Concatenate()([shared_out[:,:2], gamma_out,
                                         shared_out[:,2:]])
    model = tf.keras.Model(x, out)
    # weighted loss (3× on gamma)
    w = tf.constant([1.,1.,3.,1.,1.], dtype='float32')
    def w_mse(y,t): return tf.reduce_mean(w*tf.square(t-y), axis=-1)
    model.compile('adam', loss=w_mse)
    return model

# cosine LR schedule
def cos_decay(epoch, lr, total=50): return 1e-3 * 0.5*(1+np.cos(np.pi*epoch/total))
sched = LearningRateScheduler(cos_decay, verbose=0)
cbs = [EarlyStopping(patience=12,restore_best_weights=True),
       ReduceLROnPlateau(factor=.5,patience=6), sched]


In [8]:
# 6) Train CALL model
call_model = build_mlp(cXtr.shape[1]); call_model.fit(
    cXtr,cYtr, validation_data=(cXva,cYva),
    epochs=50, batch_size=4096, callbacks=cbs, verbose=1)


Epoch 1/50
2121/2121 [==============================] - 37s 15ms/step - loss: 0.3094 - val_loss: 0.2427 - lr: 0.0010
Epoch 2/50
2121/2121 [==============================] - 32s 15ms/step - loss: 0.1636 - val_loss: 0.3437 - lr: 9.9901e-04
Epoch 3/50
2121/2121 [==============================] - 31s 15ms/step - loss: 0.1426 - val_loss: 0.2116 - lr: 9.9606e-04
Epoch 4/50
2121/2121 [==============================] - 33s 16ms/step - loss: 0.1313 - val_loss: 0.1936 - lr: 9.9114e-04
Epoch 5/50
2121/2121 [==============================] - 31s 15ms/step - loss: 0.1217 - val_loss: 0.1219 - lr: 9.8429e-04
Epoch 6/50
2121/2121 [==============================] - 29s 14ms/step - loss: 0.1154 - val_loss: 0.1856 - lr: 9.7553e-04
Epoch 7/50
2121/2121 [==============================] - 29s 13ms/step - loss: 0.1100 - val_loss: 0.1487 - lr: 9.6489e-04
Epoch 8/50
2121/2121 [==============================] - 30s 14ms/step - loss: 0.1066 - val_loss: 0.1888 - lr: 9.5241e-04
Epoch 9/50
2121/2121 [==============

In [9]:
# 7) Train PUT model
put_model = build_mlp(pXtr.shape[1]);  put_model.fit(
    pXtr,pYtr, validation_data=(pXva,pYva),
    epochs=50, batch_size=4096, callbacks=cbs, verbose=1)


Epoch 1/50
2014/2014 [==============================] - 27s 12ms/step - loss: 0.3325 - val_loss: 0.2442 - lr: 0.0010
Epoch 2/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1857 - val_loss: 0.2479 - lr: 9.9901e-04
Epoch 3/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1616 - val_loss: 0.1909 - lr: 9.9606e-04
Epoch 4/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1476 - val_loss: 0.3475 - lr: 9.9114e-04
Epoch 5/50
2014/2014 [==============================] - 23s 12ms/step - loss: 0.1381 - val_loss: 0.2225 - lr: 9.8429e-04
Epoch 6/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1357 - val_loss: 0.1216 - lr: 9.7553e-04
Epoch 7/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1265 - val_loss: 0.2825 - lr: 9.6489e-04
Epoch 8/50
2014/2014 [==============================] - 24s 12ms/step - loss: 0.1229 - val_loss: 0.1920 - lr: 9.5241e-04
Epoch 9/50
2014/2014 [==============

In [10]:
# 8) Inverse-scale predictions
c_pred = y_scal_call.inverse_transform(call_model.predict(cXte))
c_true = y_scal_call.inverse_transform(cYte)
p_pred = y_scal_put .inverse_transform( put_model.predict(pXte))
p_true = y_scal_put .inverse_transform(pYte)

# 9) Metrics
def report(t,p,tag):
    print(f'\n{tag} MODEL'); 
    for i,g in enumerate(Y_COLS):
        print(f'{g:10s}  MSE={mean_squared_error(t[:,i],p[:,i]):.6f}  '
              f'MAE={mean_absolute_error(t[:,i],p[:,i]):.6f}  '
              f'R²={r2_score(t[:,i],p[:,i]):.4f}')
report(c_true,c_pred,'CALL'); report(p_true,p_pred,'PUT')


2631/2631 [==============================] - 9s 3ms/step

CALL MODEL
mid_price   MSE=3.210906  MAE=1.115193  R²=0.9989
delta       MSE=0.000680  MAE=0.016332  R²=0.9953
gamma       MSE=0.000425  MAE=0.003176  R²=0.9433
vega        MSE=6.931519  MAE=1.517139  R²=0.9963
theta       MSE=76.969024  MAE=2.793950  R²=0.9587

PUT MODEL
mid_price   MSE=0.801911  MAE=0.557420  R²=0.9990
delta       MSE=0.000848  MAE=0.017602  R²=0.9933
gamma       MSE=0.000281  MAE=0.004079  R²=0.9705
vega        MSE=10.755052  MAE=1.588852  R²=0.9931
theta       MSE=40.769907  MAE=2.354789  R²=0.9496
